# Import Data and Train Model


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision.transforms import ToTensor, Lambda

import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd;
import numpy as np
import math

from sklearn.model_selection import train_test_split

## Load Data: Create Dataset


### Load Absence Seizure Data


In [292]:
class AbsenceDataset(Dataset):
    def __init__(self, file_path, transform=None, target_transform=None):
        self.x, self.y = self.data_get(file_path)
        self.n_samples = len(self.x)

        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, idx):
        sample_x, sample_y = self.x[idx], self.y[idx]

        if self.transform:
            sample_x = self.transform(sample_x)

        if self.target_transform:
            sample_y = self.target_transform(sample_y)

        return sample_x, sample_y

    def __len__(self):
        return self.n_samples

    def data_get(self, file_path):
        data = pd.read_csv(file_path)
        x_loaded, y_loaded = [], []
        for index, row in data.iterrows():
            x_loaded.append(np.load(row["data"]))
            y_loaded.append(np.load(row["labels"]))

        return x_loaded, y_loaded

In [295]:
# Split Data
dataset = AbsenceDataset(
    "processed_data/absence/epochs_files.csv",
    transform=torch.tensor,
    target_transform=torch.tensor,
)

train_set, eval_set, test_set = random_split(
    dataset, [int(len(dataset) * 0.7), int(len(dataset) * 0.2), int(len(dataset) * 0.1)]
)

c:\Users\loicl\Documents\UniWork\Year 3\Dissertation\TinyML-for-Epileptic-Seizures\.venv\Lib\site-packages\torch\utils\data\dataset.py:449: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
c:\Users\loicl\Documents\UniWork\Year 3\Dissertation\TinyML-for-Epileptic-Seizures\.venv\Lib\site-packages\torch\utils\data\dataset.py:449: UserWarning: Length of split at index 2 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


In [296]:
len(dataset)

2

In [244]:
train_dataloader = DataLoader(dataset=train_set, batch_size=1, shuffle=True)
eval_dataloader = DataLoader(dataset=eval_set, batch_size=1, shuffle=True)
test_dataloader = DataLoader(dataset=test_set, batch_size=1, shuffle=True)

In [245]:
# Dummy Training Loop
n_e = 1
samples = len(dataset)
n_iter = math.ceil(samples / 4)

for e in range(n_e):
    for i, (inputs, labels) in enumerate(train_dataloader):
        print((inputs.shape))
        break

torch.Size([1, 5, 1000])


In [246]:
class AbnormalNeuralNetwork(nn.Module):
    def __init__(self, input_size=1000, output_size=1):
        super().__init__()
        self.CNN1 = (nn.Conv2d(in_channels=5, out_channels=5, kernel_size=5),)
        self.CNN2 = (nn.Conv2d(in_channels=1000, out_channels=1000, kernel_size=5),)

    def forward(self, x):
        logits = self.CNN1(x)
        logits = self.CNN2(logits)
        return logits

In [247]:
model = AbnormalNeuralNetwork().to(device)

In [248]:
# Set optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

ValueError: optimizer got an empty parameter list

In [258]:
epochs = 200  # Number of training iterations
losses = []  # Append loses to this list as training goes on

for i in range(epochs):
    for i, (x_sample, y_sample) in enumerate(train_dataloader):
        x, y = x_sample.to(device), y_sample.to(device)

        print(type(x[0][0]))

        y_pred = model(x)
        loss = loss_fn(y_pred, y)

        # Zero gradients, backward pass, update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

<class 'torch.Tensor'>


TypeError: 'tuple' object is not callable